In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

In [2]:
import sys
sys.path.append("C:/Users/w510663/Desktop/Pyri/connections/")
from conn import snowflake_conn

conn = snowflake_conn()
cur = conn.cursor()

cur.execute('''
    SELECT DISTINCT FE.CUST_ID, FE.SETID, FE.REF_DATE, FE.A_NAICS_2_DIGIT_C, FE.A_TENURE_MONTHS_N, FE.A_FLAG_REVOLVER_B, FE.A_YEARS_IN_BUSINESS_APP_N, FE.A_FLEET_SIZE_APP_N,
        FE.D_SPREAD_EXPOSURE_6M_N, FE.D_MAX_DAYS_PAST_DUE_6M_N, FE.D_MAX_EXPOSURE_1M_N, FE.D_DAYS_SINCE_LAST_SUSPENSION_CURRENT_N, FE.D_DAYS_SINCE_LAST_30DPD_CURRENT_N, 
        FE.D_DAYS_PAST_DUE_CURRENT_N, FE.D_SPREAD_FRAC_UNBILLED_6M_N, FE.D_MAX_FRAC_UNBILLED_1M_N, FE.D_MAX_DAYS_PAST_DUE_1M_N,
        FE.DNB_COMMERCIAL_CREDIT_SCORE_CURRENT_N, FE.DNB_FINANCIAL_STRESS_SCORE_CURRENT_N,
        FE.LN_LEXISNEXIS_SCORE_CURRENT_N, FE.LN_LEXISNEXIS_SBFE_SCORE_CURRENT_N,
        FE.T_TOTAL_TRX_NON_FUEL_PROPORTION_1M_N, FE.T_SPREAD_GROSS_SPEND_6M_N, FE.T_SPREAD_TOTAL_TRX_6M_N, FE.T_SUM_GALLONS_1M_N, FE.T_SPREAD_SUM_GALLONS_6M_N,
        FE.FLAG_60_DPD_366_DAYS,
        FL.EXCL_PORTFOLIO_FLAG, FL.EXCL_CUST_STATUS_FLAG, FL.EXCL_CUST_TENURE_FLAG, FL.EXCL_GOV_VIP_FLAG, FL.EXCL_DUMMY_CREDIT_LIMIT_FLAG, FL.EXCL_LARGE_CREDIT_LIMIT_FLAG,
        FL.EXCL_MATERIAL_DELINQUENCY_FLAG, FL.EXCL_CUST_CONVERSION_FLAG, FL.EXCL_FRAUD_FLAG, FL.EXCL_EFPS_FLAG,
        L0.WX_CUST_SUSP_CLASS, L0.GOVERNMENT_FLG
        -- P.MODEL_PROB
    FROM DATAIKU_RAW_SUB.RISK_FRAUD.PM_FEATURE_TABLE FE
    LEFT JOIN DATAIKU_RAW_SUB.RISK_FRAUD.PM_STAGE_L1_CUSTOMER_FILTER_FLAGS_PERSIST FL
        ON FE.CUST_ID = FL.CUST_ID AND FE.REF_DATE = FL.REF_DATE AND FE.SETID = FL.SETID
    LEFT JOIN DATAIKU_RAW_SUB.RISK_FRAUD.PM_STAGE_L0_CUSTOMER_FILTER_ATTRIBUTES_PERSIST L0
        ON FE.CUST_ID = L0.CUST_ID AND FE.REF_DATE = L0.REF_DATE AND FE.SETID = L0.SETID
    -- LEFT JOIN DATAIKU_RAW_SUB.RISK_FRAUD.PM_NAF_OUTPUT_RISK_MODEL P --SHAP for each feature can be extract here
    -- ON FE.CUST_ID = P.CUST_ID AND FE.REF_DATE = P.REF_DATE AND FE.SETID = P.SETID
    WHERE FE.REF_DATE >= '2021-01-01' AND FE.REF_DATE <= '2022-08-01'
    -- AND GREATEST(FL.EXCL_PORTFOLIO_FLAG, FL.EXCL_CUST_STATUS_FLAG, FL.EXCL_CUST_TENURE_FLAG, FL.EXCL_DUMMY_CREDIT_LIMIT_FLAG, FL.EXCL_LARGE_CREDIT_LIMIT_FLAG,
    --  FL.EXCL_MATERIAL_DELINQUENCY_FLAG, FL.EXCL_CUST_CONVERSION_FLAG, FL.EXCL_FRAUD_FLAG, FL.EXCL_EFPS_FLAG) = 0
;
''')
all_acct = pd.DataFrame.from_records(iter(cur), columns=[x[0] for x in cur.description])
all_acct.shape

c:\Users\w510663\Anaconda3\lib\site-packages\snowflake\connector\options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (11.0.0), please install a version that adheres to: 'pyarrow<10.1.0,>=10.0.1; extra == "pandas"'
  warn_incompatible_dep(
Failed to import ArrowResult. No Apache Arrow result set format can be used. ImportError: DLL load failed while importing arrow_iterator: The specified procedure could not be found.


(7008202, 39)

### **Accounts with GOV / VIP Flag** - WITH and WITHOUT Other Flags

In [37]:
df_gov_vip = all_acct.copy()

# Create a column called 'EXCL_CRITERIA_SUM' to sum up all exclusion criteria
df_gov_vip['EXCL_CRITERIA_SUM'] = \
    df_gov_vip.EXCL_PORTFOLIO_FLAG \
    + df_gov_vip.EXCL_CUST_STATUS_FLAG \
    + df_gov_vip.EXCL_CUST_TENURE_FLAG \
    + df_gov_vip.EXCL_GOV_VIP_FLAG \
    + df_gov_vip.EXCL_DUMMY_CREDIT_LIMIT_FLAG \
    + df_gov_vip.EXCL_LARGE_CREDIT_LIMIT_FLAG \
    + df_gov_vip.EXCL_MATERIAL_DELINQUENCY_FLAG \
    + df_gov_vip.EXCL_CUST_CONVERSION_FLAG \
    + df_gov_vip.EXCL_FRAUD_FLAG \
    + df_gov_vip.EXCL_EFPS_FLAG

# Create a column called 'EXCL_GOV_VIP_FLAG_ONLY' where EXCL_GOV_VIP_FLAG = 1 and EXCL_CRITERIA_SUM = 1
df_gov_vip['EXCL_GOV_VIP_FLAG_ONLY'] = np.where((df_gov_vip.EXCL_GOV_VIP_FLAG == 1) & (df_gov_vip.EXCL_CRITERIA_SUM == 1), 1, 0)

# Create a column called 'EXCL_ONLY_GOV_VIP_FLAG' where EXCL_GOV_VIP_FLAG = 1 and EXCL_CRITERIA_SUM > 1
df_gov_vip['EXCL_GOV_VIP_FLAG_WITH_OTHERS'] = np.where((df_gov_vip.EXCL_GOV_VIP_FLAG == 1) & (df_gov_vip.EXCL_CRITERIA_SUM > 1), 1, 0)

# Create a dataframe for CUST_ID count for each ref_date for EXCL_GOV_VIP_FLAG_ONLY and EXCL_GOV_VIP_FLAG_WITH_OTHERS
df_gov_vip = df_gov_vip.groupby('REF_DATE').agg({'CUST_ID': 'nunique', 'EXCL_GOV_VIP_FLAG_ONLY': 'sum', 'EXCL_GOV_VIP_FLAG_WITH_OTHERS': 'sum'}).reset_index()
df_gov_vip['EXCL_GOV_VIP_FLAG_ONLY_PERCENT'] = df_gov_vip['EXCL_GOV_VIP_FLAG_ONLY'] / df_gov_vip['CUST_ID']
df_gov_vip['EXCL_GOV_VIP_FLAG_WITH_OTHERS_PERCENT'] = df_gov_vip['EXCL_GOV_VIP_FLAG_WITH_OTHERS'] / df_gov_vip['CUST_ID']

df_gov_vip['IN_MODEL_CUST'] = df_gov_vip['CUST_ID'] - df_gov_vip['EXCL_GOV_VIP_FLAG_ONLY'] - df_gov_vip['EXCL_GOV_VIP_FLAG_WITH_OTHERS']

df_gov_vip = df_gov_vip[['REF_DATE', 'CUST_ID', 'IN_MODEL_CUST', 'EXCL_GOV_VIP_FLAG_ONLY', 'EXCL_GOV_VIP_FLAG_ONLY_PERCENT', 'EXCL_GOV_VIP_FLAG_WITH_OTHERS', 'EXCL_GOV_VIP_FLAG_WITH_OTHERS_PERCENT']]
df_gov_vip

,REF_DATE,CUST_ID,IN_MODEL_CUST,EXCL_GOV_VIP_FLAG_ONLY,EXCL_GOV_VIP_FLAG_ONLY_PERCENT,EXCL_GOV_VIP_FLAG_WITH_OTHERS,EXCL_GOV_VIP_FLAG_WITH_OTHERS_PERCENT
0,2021-03-01,520483,487735,14847,0.028525,17901,0.034393
1,2021-06-01,537163,503964,14971,0.027870,18228,0.033934
2,2021-09-01,555096,521297,15431,0.027799,18368,0.033090
3,2021-12-01,579898,545989,15970,0.027539,17939,0.030935
4,2022-01-01,585767,551700,16054,0.027407,18013,0.030751
5,2022-02-01,590508,556140,16014,0.027119,18354,0.031082
6,2022-03-01,597779,562397,15977,0.026727,19405,0.032462
7,2022-04-01,601395,566623,15480,0.025740,19292,0.032079
8,2022-05-01,603232,567620,16450,0.027270,19162,0.031766
9,2022-06-01,606192,570509,17330,0.028588,18353,0.030276


### **Accounts with GOV / VIP Flag Only** - NOT with other exclusion flags

In [40]:
df_wx_cust_susp_class = all_acct.copy()

df_wx_cust_susp_class['EXCL_CRITERIA_SUM'] = \
    df_wx_cust_susp_class.EXCL_PORTFOLIO_FLAG \
    + df_wx_cust_susp_class.EXCL_CUST_STATUS_FLAG \
    + df_wx_cust_susp_class.EXCL_CUST_TENURE_FLAG \
    + df_wx_cust_susp_class.EXCL_GOV_VIP_FLAG \
    + df_wx_cust_susp_class.EXCL_DUMMY_CREDIT_LIMIT_FLAG \
    + df_wx_cust_susp_class.EXCL_LARGE_CREDIT_LIMIT_FLAG \
    + df_wx_cust_susp_class.EXCL_MATERIAL_DELINQUENCY_FLAG \
    + df_wx_cust_susp_class.EXCL_CUST_CONVERSION_FLAG \
    + df_wx_cust_susp_class.EXCL_FRAUD_FLAG \
    + df_wx_cust_susp_class.EXCL_EFPS_FLAG

# Create a column called 'EXCL_GOV_VIP_FLAG_ONLY' where EXCL_GOV_VIP_FLAG = 1 and EXCL_CRITERIA_SUM = 1
df_wx_cust_susp_class['EXCL_GOV_FLAG_ONLY'] = np.where((df_wx_cust_susp_class.EXCL_GOV_VIP_FLAG == 1) 
                                                       & ((df_wx_cust_susp_class.WX_CUST_SUSP_CLASS == 'G') | (df_wx_cust_susp_class.GOVERNMENT_FLG == 'Y'))
                                                       & (df_wx_cust_susp_class.EXCL_CRITERIA_SUM == 1), 1, 0)

df_wx_cust_susp_class['EXCL_VIP_FLAG_ONLY'] = np.where((df_wx_cust_susp_class.EXCL_GOV_VIP_FLAG == 1) 
                                                       & ((df_wx_cust_susp_class.WX_CUST_SUSP_CLASS == 'V') & (df_wx_cust_susp_class.GOVERNMENT_FLG != 'Y'))
                                                       & (df_wx_cust_susp_class.EXCL_CRITERIA_SUM == 1), 1, 0)

df_wx_cust_susp_class['EXCL_E_FLAG_ONLY'] = np.where((df_wx_cust_susp_class.EXCL_GOV_VIP_FLAG == 1) 
                                                       & ((df_wx_cust_susp_class.WX_CUST_SUSP_CLASS == 'E') & (df_wx_cust_susp_class.GOVERNMENT_FLG == 'Y'))
                                                       & (df_wx_cust_susp_class.EXCL_CRITERIA_SUM == 1), 1, 0)

df_wx_cust_susp_class['EXCL_K_FLAG_ONLY'] = np.where((df_wx_cust_susp_class.EXCL_GOV_VIP_FLAG == 1) 
                                                       & ((df_wx_cust_susp_class.WX_CUST_SUSP_CLASS == 'K') & (df_wx_cust_susp_class.GOVERNMENT_FLG == 'Y'))
                                                       & (df_wx_cust_susp_class.EXCL_CRITERIA_SUM == 1), 1, 0)

df_wx_cust_susp_class = df_wx_cust_susp_class.groupby('REF_DATE').agg({'CUST_ID': 'nunique'
                                                                       , 'EXCL_GOV_FLAG_ONLY': 'sum'
                                                                       , 'EXCL_VIP_FLAG_ONLY': 'sum'
                                                                       , 'EXCL_E_FLAG_ONLY': 'sum'
                                                                       , 'EXCL_K_FLAG_ONLY': 'sum'}).reset_index()
df_wx_cust_susp_class

,REF_DATE,CUST_ID,EXCL_GOV_FLAG_ONLY,EXCL_VIP_FLAG_ONLY,EXCL_E_FLAG_ONLY,EXCL_K_FLAG_ONLY
0,2021-03-01,520483,12683,117,10,3881
1,2021-06-01,537163,12778,118,9,3895
2,2021-09-01,555096,13141,114,9,4104
3,2021-12-01,579898,13607,116,10,4310
4,2022-01-01,585767,13698,120,11,4356
5,2022-02-01,590508,13648,115,10,4377
6,2022-03-01,597779,13605,112,11,4402
7,2022-04-01,601395,13802,121,6,3235
8,2022-05-01,603232,13978,122,11,4527
9,2022-06-01,606192,14047,915,11,4616


In [11]:
df = all_acct.copy()